# Data Wrangling with DataFrames Coding Quiz

Use this Jupyter notebook to find the answers to the quiz in the previous section. There is an answer key in the next part of the lesson.

In [84]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import isnan, count, when, col, desc, udf, col, sort_array, asc, avg
from pyspark.sql.functions import sum as Fsum
from pyspark.sql.window import Window
from pyspark.sql.types import IntegerType

# TODOS: 
# 1) import any other libraries you might need
# 2) instantiate a Spark session 
# 3) read in the data set located at the path "data/sparkify_log_small.json"
# 4) write code to answer the quiz questions 

In [2]:
spark = SparkSession \
    .builder \
    .appName("Wrangling Data") \
    .getOrCreate()

In [3]:
path = "data/sparkify_log_small.json"
user_log = spark.read.json(path)

In [4]:
user_log.printSchema()

root
 |-- artist: string (nullable = true)
 |-- auth: string (nullable = true)
 |-- firstName: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- itemInSession: long (nullable = true)
 |-- lastName: string (nullable = true)
 |-- length: double (nullable = true)
 |-- level: string (nullable = true)
 |-- location: string (nullable = true)
 |-- method: string (nullable = true)
 |-- page: string (nullable = true)
 |-- registration: long (nullable = true)
 |-- sessionId: long (nullable = true)
 |-- song: string (nullable = true)
 |-- status: long (nullable = true)
 |-- ts: long (nullable = true)
 |-- userAgent: string (nullable = true)
 |-- userId: string (nullable = true)



In [7]:
user_log.userId

Column<b'userId'>

# Question 1

Which page did user id "" (empty string) NOT visit?

In [5]:
# TODO: write your code to answer question 1

In [47]:
## Getting all the pages
allPages_DF = user_log.select('page').dropDuplicates()
allPages_DF.show()

+----------------+
|            page|
+----------------+
|Submit Downgrade|
|            Home|
|       Downgrade|
|          Logout|
|   Save Settings|
|           About|
|        Settings|
|           Login|
|        NextSong|
|            Help|
|         Upgrade|
|           Error|
|  Submit Upgrade|
+----------------+



In [52]:
## The user didn't visit 
emptyId_DF = user_log.select("page").filter(user_log.userId=='').dropDuplicates()
emptyId_DF.show()

+-----+
| page|
+-----+
| Home|
|About|
|Login|
| Help|
+-----+



In [58]:
for page_not_visited in set(allPages_DF.collect()) - set(emptyId_DF.collect()):
    print(page_not_visited.page)

Save Settings
NextSong
Submit Upgrade
Error
Submit Downgrade
Upgrade
Settings
Downgrade
Logout


# Question 2 - Reflect

What type of user does the empty string user id most likely refer to?


In [59]:
# TODO: use this space to explore the behavior of the user with an empty string
user_log.select("*").filter(user_log.userId == "").groupby(user_log.sessionId).count().show()
## Probably guest users who haven't signed in yet or are about to sign in. 

+---------+-----+
|sessionId|count|
+---------+-----+
|     4894|    1|
|     2941|    1|
|     6892|    2|
|     6362|    2|
|     4108|    1|
|     5915|    1|
|     6605|    2|
|     5364|    1|
|     2612|    1|
|     2966|    1|
|     5690|    2|
|      184|    1|
|     5219|    2|
|     1540|    2|
|     7096|    1|
|     5826|    2|
|     2080|    1|
|     1895|    1|
|     1064|    1|
|     5809|    1|
+---------+-----+
only showing top 20 rows



# Question 3

How many female users do we have in the data set?

In [67]:
# TODO: write your code to answer question 3
## investigate how it filters. It's probably based on key, but what if I don't mention the key? 
## how does it detect the duplication drop?
user_log.select('userId','gender').filter(user_log.gender == 'F').dropDuplicates().count()

462

# Question 4

How many songs were played from the most played artist?

In [110]:
# TODO: write your code to answer question 4
mostPlayed_1 = user_log.select("song","artist").groupby(user_log.artist).count().sort(desc("count")) \
.filter(user_log.artist != "null").take(1)
print(mostPlayed_1)

[Row(artist='Coldplay', count=83)]


# Question 5 (challenge)

How many songs do users listen to on average between visiting our home page? Please round your answer to the closest integer.



In [147]:
# TODO: write your code to answer question 5
function = udf(lambda ishome : int(ishome == 'Home'), IntegerType())
convert_to_int = udf(lambda x: round(x))

user_window = Window \
            .partitionBy('userId') \
            .orderBy(desc('ts')) \
            .rangeBetween(Window.unboundedPreceding, 0)

cusum = user_log.filter((user_log.page == 'NextSong') | (user_log.page == 'Home')) \
    .select('userID', 'page', 'ts') \
    .withColumn('homevisit', function(col('page'))) \
    .withColumn('period', Fsum('homevisit').over(user_window))

cusum.filter((cusum.page == 'NextSong')) \
    .groupBy('userID', 'period') \
    .agg({'period':'count'}) \
    .agg({'count(period)':'avg'})\
    .withColumn('avg(count(period))',convert_to_int(col("avg(count(period))")))\
    .show()

+------------------+
|avg(count(period))|
+------------------+
|                 7|
+------------------+

